<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [3]:
d_tweets=d_tweets_dataset['tweet']

In [4]:
non_d_tweets=non_d_tweets_dataset['tweet']

In [5]:
y1=np.ones((3082,1),dtype="int")

In [6]:
y2=np.zeros((2505,1),dtype="int")

In [7]:
y=np.concatenate((y1,y2),axis=0)

In [8]:
res=pd.DataFrame(y)

In [9]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [10]:
txt.shape
txt=txt.reset_index(drop="true")

In [11]:
df=pd.concat([txt,res],axis=1,join="inner")

In [12]:
df.columns=["text","result"]


In [15]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

In [17]:
from gensim.models import FastText

# Train FastText model
fasttext_model = FastText(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [18]:
# Get the FastText word vectors
word_vectors = fasttext_model.wv


In [ ]:
# Convert text to FastText word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100)
# X_fasttext_padded
# y = df['result'].values


In [ ]:
# Prepare target variable y (same as before)

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:
X_fasttext = []
for sentence in tokenized_text:
    embedding = []
    for word in sentence:
        if word in word_vectors:
            embedding.append(word_vectors[word])
    X_fasttext.append(embedding)

# Find the maximum sequence length in X_fasttext
max_sequence_length = max(len(embedding) for embedding in X_fasttext)

# Pad sequences to the maximum length
X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100, dtype='float32', padding='post')
X_fasttext_padded.shape
y = df['result'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext_padded, y, test_size=0.2, random_state=42)

In [ ]:
# Define the LSTM-RNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(100, 100)))

# Add more layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer for regularization

# # Add another LSTM layer with 64 units, dropout, and recurrent dropout
# model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))

# # Add more layers
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))  # Dropout layer for regularization

# Output layer with a single unit and sigmoid activation function for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Step 7: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 8: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')
# Step 9: Make Predictions (same as before)


Epoch 1/10
  8/140 [>.............................] - ETA: 52s - loss: 0.6918 - accuracy: 0.5234

KeyboardInterrupt: ignored

Using Glove

In [ ]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


OSError: ignored

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_md

2023-11-13 11:32:13.871726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 11:32:13.871818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 11:32:13.871870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 11:32:15.566794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


In [ ]:
# Convert text to GloVe word embeddings
X_glove = []
for sentence in tokenized_text:
    embedding = [get_word_vector(word) for word in sentence]
    embedding = [word_embedding for word_embedding in embedding if word_embedding is not None]
    X_glove.append(embedding)

# Find the maximum sequence length in X_glove
max_sequence_length = max(len(embedding) for embedding in X_glove)

# Pad sequences to the maximum length
X_glove_padded = pad_sequences(X_glove, maxlen=max_sequence_length, dtype='float32', padding='post')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_glove_padded, y, test_size=0.2, random_state=42)


In [ ]:
# Define the LSTM-RNN Model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_sequence_length, 300)))  # 300 is the dimensionality of GloVe vectors
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# # Step 6: Make Predictions
# predictions = model.predict_classes(X_test)


Epoch 1/10


KeyboardInterrupt: ignored

Using TF-IDF Vectorization

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
X = df['text']
y = df['result']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.1)


In [ ]:
y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)  # Apply thresholding at 0.5 for binary classification

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Using BiLSTM and word2vec

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
max_words = 10000  # Choose based on your dataset
maxlen = 100

In [ ]:

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

In [ ]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 6236 unique tokens.


In [ ]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

In [ ]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

Found 6236 unique tokens.


In [ ]:
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
embedding_dim = 100  # Should match the vector_size in Word2Vec model

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict_classes(x_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Epoch 1/10
140/140 [==============================] - 36s 188ms/step - loss: 0.4552 - accuracy: 0.7776 - val_loss: 0.3082 - val_accuracy: 0.8605
Epoch 2/10
140/140 [==============================] - 23s 164ms/step - loss: 0.1970 - accuracy: 0.9224 - val_loss: 0.2939 - val_accuracy: 0.8757
Epoch 3/10
140/140 [==============================] - 25s 178ms/step - loss: 0.1098 - accuracy: 0.9595 - val_loss: 0.3399 - val_accuracy: 0.8784
Epoch 4/10
140/140 [==============================] - 29s 207ms/step - loss: 0.0664 - accuracy: 0.9752 - val_loss: 0.3848 - val_accuracy: 0.8873
Epoch 5/10
140/140 [==============================] - 27s 189ms/step - loss: 0.0457 - accuracy: 0.9828 - val_loss: 0.4635 - val_accuracy: 0.8846
Epoch 6/10
140/140 [==============================] - 22s 155ms/step - loss: 0.0346 - accuracy: 0.9877 - val_loss: 0.5085 - val_accuracy: 0.8927
Epoch 7/10
140/140 [==============================] - 25s 178ms/step - loss: 0.0296 - accuracy: 0.9886 - val_loss: 0.4992 - val_ac

AttributeError: ignored

Using BiLSTM and fasttext

In [ ]:
import pandas as pd
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Build the BiLSTM model
embedding_dim = 100  # Should match the vector_size in FastText model

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict_classes(x_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 32s 180ms/step - loss: 0.4612 - accuracy: 0.7644 - val_loss: 0.2984 - val_accuracy: 0.8515
Epoch 2/10
140/140 [==============================] - 22s 158ms/step - loss: 0.1958 - accuracy: 0.9217 - val_loss: 0.2715 - val_accuracy: 0.8900
Epoch 3/10
140/140 [==============================] - 24s 172ms/step - loss: 0.1050 - accuracy: 0.9617 - val_loss: 0.3441 - val_accuracy: 0.8730
Epoch 4/10
140/140 [==============================] - 23s 163ms/step - loss: 0.0656 - accuracy: 0.9776 - val_loss: 0.4310 - val_accuracy: 0.8855
Epoch 5/10
140/140 [==============================] - 24s 174ms/step - loss: 0.0380 - accuracy: 0.9877 - val_loss: 0.5228 - val_accuracy: 0.8748
Epoch 6/10
140/140 [==============================] - 24s 169ms/step - loss: 0.0313 - accuracy: 0.9868 - val_loss: 0.5554 - val_accuracy: 0.8730
Epoch 7/10
140/140 [==============================] - 24s 170ms/step - loss: 0.0243 - accuracy: 0.9915 -

AttributeError: ignored

Using BiLSTM and Glove

In [ ]:
!pip install glove-python

In [ ]:
!pip install glove_python

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Load the pre-trained GloVe embeddings
glove_file = '/content/glove.6B.100d.txt'  # Change this to the actual path of your downloaded GloVe file
embedding_dim = 100  # Should match the dimensionality of your pre-trained GloVe vectors

# Create an embedding matrix
embedding_matrix = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words and word in embedding_matrix:
        embedding_matrix[i] = embedding_matrix[word]

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict_classes(x_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')



Found 6236 unique tokens.


<ipython-input-22-acfa59089f09>:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i < max_words and word in embedding_matrix:


Epoch 1/10
140/140 [==============================] - 23s 127ms/step - loss: 0.6882 - accuracy: 0.5491 - val_loss: 0.6926 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 19s 139ms/step - loss: 0.6884 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 19s 135ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 17s 123ms/step - loss: 0.6884 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 17s 121ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 17s 124ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6891 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 18s 125ms/step - loss: 0.6877 - accuracy: 0.5531 - val_loss: 0.6894 - val_ac

AttributeError: ignored

In [23]:
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

35/35 [==============================] - 6s 66ms/step
Validation Accuracy: 0.5456171735241503


Using BiLSTM and tf-idf

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# TF-IDF Vectorization
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tfidf_vectorizer = TfidfVectorizer(max_features=max_words)
tfidf_train = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_val = tfidf_vectorizer.transform(val_data['text'])

# Padding sequences
x_train = pad_sequences(tfidf_train.toarray(), maxlen=maxlen)
x_val = pad_sequences(tfidf_val.toarray(), maxlen=maxlen)

y_train = train_data['result'].values
y_val = val_data['result'].values

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))




Epoch 1/10
140/140 [==============================] - 31s 184ms/step - loss: 0.6895 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 22s 158ms/step - loss: 0.6881 - accuracy: 0.5531 - val_loss: 0.6892 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 26s 187ms/step - loss: 0.6884 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 23s 168ms/step - loss: 0.6880 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 5/10
140/140 [==============================] - 24s 170ms/step - loss: 0.6880 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 6/10
140/140 [==============================] - 27s 196ms/step - loss: 0.6876 - accuracy: 0.5531 - val_loss: 0.6902 - val_accuracy: 0.5456
Epoch 7/10
140/140 [==============================] - 35s 253ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6892 - val_ac

AttributeError: ignored

In [21]:
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

35/35 [==============================] - 6s 66ms/step
Validation Accuracy: 0.5456171735241503


Using BERT and word2vec

In [24]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [25]:
!pip install gensim

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel, BertConfig
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, Concatenate

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='np')

# Load pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
with torch.no_grad():
    bert_outputs = bert_model(**bert_input)

# Extract embeddings from BERT output
bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings, axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


NameError: ignored

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Load pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
with torch.no_grad():
    bert_outputs = bert_model(**bert_input)

# Extract embeddings from BERT output
bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')




KeyError: ignored

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Check if running on Google Colab and TPU is available
if 'COLAB_TPU_ADDR' in os.environ:
    # Enable TPU acceleration
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    strategy = None

# Load pre-trained BERT model
with strategy.scope() if strategy else nullcontext():
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    # Generate BERT embeddings
    with torch.no_grad():
        bert_outputs = bert_model(**bert_input)

    # Extract embeddings from BERT output
    bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Concatenate, Embedding, GlobalAveragePooling1D
import torch
import os
from contextlib import contextmanager

# Define nullcontext for Python versions < 3.7
@contextmanager
def nullcontext(enter_result=None):
    yield enter_result

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize using BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert sentences to BERT input format
bert_input = bert_tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Check if running on Google Colab and TPU is available
if 'COLAB_TPU_ADDR' in os.environ:
    # Enable TPU acceleration
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    strategy = nullcontext()

# Load pre-trained BERT model
with strategy:
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    # Generate BERT embeddings
    with torch.no_grad():
        bert_outputs = bert_model(**bert_input)

    # Extract embeddings from BERT output
    bert_embeddings = bert_outputs['last_hidden_state']

# Average pooling over the sequence dimension
bert_avg_pooling = np.mean(bert_embeddings.numpy(), axis=1)

# Combine Word2Vec and BERT embeddings
combined_embeddings = np.concatenate([word2vec_model.wv[word] for word in df['text']], axis=0)
combined_embeddings = np.concatenate([combined_embeddings, bert_avg_pooling], axis=1)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=combined_embeddings.shape[1]))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')




Found 6236 unique tokens.


Using GRU and Word2vec

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Use a GRU layer for text representation
embedding_dim = 100  # Adjust based on your Word2Vec model dimension

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(GRU(64))  # You can adjust the number of units based on your needs
# model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 35s 186ms/step - loss: 0.4808 - accuracy: 0.7471 - val_loss: 0.3343 - val_accuracy: 0.8551
Epoch 2/10
140/140 [==============================] - 23s 166ms/step - loss: 0.1907 - accuracy: 0.9255 - val_loss: 0.2826 - val_accuracy: 0.8819
Epoch 3/10
140/140 [==============================] - 15s 105ms/step - loss: 0.0964 - accuracy: 0.9638 - val_loss: 0.3315 - val_accuracy: 0.8801
Epoch 4/10
140/140 [==============================] - 14s 103ms/step - loss: 0.0601 - accuracy: 0.9763 - val_loss: 0.3961 - val_accuracy: 0.8819
Epoch 5/10
140/140 [==============================] - 15s 109ms/step - loss: 0.0388 - accuracy: 0.9864 - val_loss: 0.4635 - val_accuracy: 0.8748
Epoch 6/10
140/140 [==============================] - 13s 96ms/step - loss: 0.0234 - accuracy: 0.9913 - val_loss: 0.5974 - val_accuracy: 0.8766
Epoch 7/10
140/140 [==============================] - 13s 90ms/step - loss: 0.0236 - accuracy: 0.9910 - v

Using BiGRU and word2vec

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your Word2Vec model
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Adjust based on your Word2Vec model dimension

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 36s 186ms/step - loss: 0.4836 - accuracy: 0.7407 - val_loss: 0.2954 - val_accuracy: 0.8703
Epoch 2/10
140/140 [==============================] - 36s 257ms/step - loss: 0.1993 - accuracy: 0.9219 - val_loss: 0.2821 - val_accuracy: 0.8927
Epoch 3/10
140/140 [==============================] - 21s 150ms/step - loss: 0.1137 - accuracy: 0.9584 - val_loss: 0.3104 - val_accuracy: 0.8927
Epoch 4/10
140/140 [==============================] - 22s 154ms/step - loss: 0.0705 - accuracy: 0.9767 - val_loss: 0.3587 - val_accuracy: 0.8864
Epoch 5/10
140/140 [==============================] - 23s 167ms/step - loss: 0.0420 - accuracy: 0.9850 - val_loss: 0.4464 - val_accuracy: 0.8864
Epoch 6/10
140/140 [==============================] - 23s 163ms/step - loss: 0.0363 - accuracy: 0.9888 - val_loss: 0.4981 - val_accuracy: 0.8864
Epoch 7/10
140/140 [==============================] - 22s 155ms/step - loss: 0.0267 - accuracy: 0.9890 -

Using BiGRU and fasttext

In [17]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=212eaad7fd5ed77ee8745b2fd69ec43fa3fa8bd3cec301b0c07d7174be93ecdf
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU
import tensorflow as tf

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = fasttext.train_unsupervised(sentences, model='skipgram', dim=100, epoch=10)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the FastText embedding dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with FastText embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


TypeError: ignored

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the FastText embedding dimension

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with FastText embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = fasttext_model.wv[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 36s 187ms/step - loss: 0.5761 - accuracy: 0.6959 - val_loss: 0.4978 - val_accuracy: 0.7630
Epoch 2/10
140/140 [==============================] - 18s 126ms/step - loss: 0.4995 - accuracy: 0.7574 - val_loss: 0.4944 - val_accuracy: 0.7657
Epoch 3/10
140/140 [==============================] - 18s 128ms/step - loss: 0.4945 - accuracy: 0.7577 - val_loss: 0.4708 - val_accuracy: 0.7800
Epoch 4/10
140/140 [==============================] - 17s 125ms/step - loss: 0.4819 - accuracy: 0.7659 - val_loss: 0.4680 - val_accuracy: 0.7826
Epoch 5/10
140/140 [==============================] - 18s 126ms/step - loss: 0.4768 - accuracy: 0.7675 - val_loss: 0.4664 - val_accuracy: 0.7853
Epoch 6/10
140/140 [==============================] - 24s 169ms/step - loss: 0.4693 - accuracy: 0.7744 - val_loss: 0.4592 - val_accuracy: 0.7907
Epoch 7/10
140/140 [==============================] - 18s 127ms/step - loss: 0.4698 - accuracy: 0.7706 -

Using BiGRU and Glove

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Download GloVe embeddings (you can choose a different dimension, e.g., 50, 100, 200, 300)
glove_file = 'glove.6B.100d.txt'
glove_w2v_file = 'glove.6B.100d.w2v.txt'

# Convert GloVe format to Word2Vec format
glove2word2vec(glove_file, glove_w2v_file)

# Load the GloVe Word2Vec model
glove_model = KeyedVectors.load_word2vec_format(glove_w2v_file, binary=False)

# Use a Bidirectional GRU layer for text representation
embedding_dim = 100  # Should match the GloVe embedding dimension (in this case, 100)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with GloVe embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = glove_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Found 6236 unique tokens.


<ipython-input-20-59bfb56fed76>:35: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, glove_w2v_file)


Epoch 1/10
140/140 [==============================] - 23s 129ms/step - loss: 0.4539 - accuracy: 0.7807 - val_loss: 0.3334 - val_accuracy: 0.8399
Epoch 2/10
140/140 [==============================] - 17s 124ms/step - loss: 0.3244 - accuracy: 0.8599 - val_loss: 0.3067 - val_accuracy: 0.8631
Epoch 3/10
140/140 [==============================] - 26s 189ms/step - loss: 0.3086 - accuracy: 0.8666 - val_loss: 0.3107 - val_accuracy: 0.8551
Epoch 4/10
140/140 [==============================] - 31s 222ms/step - loss: 0.2751 - accuracy: 0.8854 - val_loss: 0.2936 - val_accuracy: 0.8730
Epoch 5/10
140/140 [==============================] - 18s 126ms/step - loss: 0.2636 - accuracy: 0.8868 - val_loss: 0.2810 - val_accuracy: 0.8846
Epoch 6/10
140/140 [==============================] - 17s 124ms/step - loss: 0.2414 - accuracy: 0.8982 - val_loss: 0.2830 - val_accuracy: 0.8873
Epoch 7/10
140/140 [==============================] - 17s 125ms/step - loss: 0.2220 - accuracy: 0.9071 - val_loss: 0.2731 - val_ac

In [22]:


model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen, trainable=False))
model.add(Bidirectional(GRU(64)))  # You can adjust the number of units based on your needs
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Set the embedding matrix with GloVe embeddings
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_matrix[i] = glove_model[word]
        except KeyError:
            pass

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Epoch 1/10
140/140 [==============================] - 36s 187ms/step - loss: 0.4663 - accuracy: 0.7722 - val_loss: 0.3473 - val_accuracy: 0.8399
Epoch 2/10
140/140 [==============================] - 18s 128ms/step - loss: 0.3295 - accuracy: 0.8599 - val_loss: 0.3019 - val_accuracy: 0.8640
Epoch 3/10
140/140 [==============================] - 22s 157ms/step - loss: 0.2990 - accuracy: 0.8725 - val_loss: 0.2966 - val_accuracy: 0.8757
Epoch 4/10
140/140 [==============================] - 22s 154ms/step - loss: 0.2848 - accuracy: 0.8789 - val_loss: 0.2807 - val_accuracy: 0.8846
Epoch 5/10
140/140 [==============================] - 29s 207ms/step - loss: 0.2590 - accuracy: 0.8908 - val_loss: 0.3446 - val_accuracy: 0.8587
Epoch 6/10
140/140 [==============================] - 25s 174ms/step - loss: 0.2378 - accuracy: 0.8991 - val_loss: 0.2778 - val_accuracy: 0.8891
Epoch 7/10
140/140 [==============================] - 18s 130ms/step - loss: 0.2188 - accuracy: 0.9103 - val_loss: 0.3033 - val_ac

Using BiGRU and tf-idf

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, GRU

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = TfidfVectorizer(max_features=max_words)
tfidf_matrix = tokenizer.fit_transform(df['text']).toarray()

# Use a Bidirectional GRU layer for text representation
embedding_dim = tfidf_matrix.shape[1]  # Use the number of features from TF-IDF

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=embedding_dim))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(tfidf_matrix, df['result'].values, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred_probs = model.predict(x_val)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')


Epoch 1/10
140/140 [==============================] - 6s 28ms/step - loss: 0.5767 - accuracy: 0.7389 - val_loss: 0.4321 - val_accuracy: 0.8524
Epoch 2/10
140/140 [==============================] - 3s 22ms/step - loss: 0.3188 - accuracy: 0.9112 - val_loss: 0.3109 - val_accuracy: 0.8649
Epoch 3/10
140/140 [==============================] - 2s 16ms/step - loss: 0.2005 - accuracy: 0.9427 - val_loss: 0.2796 - val_accuracy: 0.8685
Epoch 4/10
140/140 [==============================] - 2s 14ms/step - loss: 0.1423 - accuracy: 0.9604 - val_loss: 0.2699 - val_accuracy: 0.8748
Epoch 5/10
140/140 [==============================] - 2s 18ms/step - loss: 0.1067 - accuracy: 0.9693 - val_loss: 0.2807 - val_accuracy: 0.8685
Epoch 6/10
140/140 [==============================] - 2s 17ms/step - loss: 0.0833 - accuracy: 0.9781 - val_loss: 0.2894 - val_accuracy: 0.8757
Epoch 7/10
140/140 [==============================] - 3s 23ms/step - loss: 0.0670 - accuracy: 0.9828 - val_loss: 0.2911 - val_accuracy: 0.8784